### Naive Bayes

In [2]:
import numpy as np
import pandas as pd

In [3]:
### 构造数据集
### 来自于李航统计学习方法表4.1
x1 = [1,1,1,1,1,2,2,2,2,2,3,3,3,3,3]
x2 = ['S','M','M','S','S','S','M','M','L','L','L','M','M','L','L']
y = [-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1]

df = pd.DataFrame({'x1':x1, 'x2':x2, 'y':y})
df.head()

,x1,x2,y
0,1,S,-1
1,1,M,-1
2,1,M,1
3,1,S,1
4,1,S,-1


In [4]:
X = df[['x1', 'x2']]
y = df[['y']]

In [5]:
y[y.columns[0]].value_counts()

 1    9
-1    6
Name: y, dtype: int64

In [6]:
X.columns

Index(['x1', 'x2'], dtype='object')

In [7]:
def nb_fit(X, y):
    '''
    输入：
    X，y，为pandas数据框格式，输入，标签
    输出：
    classes：标签类别
    class_prior：类先验概率分布
    class_condition：类条件概率分布
    '''
    #标签类别
    classes = y[y.columns[0]].unique()
    #标签类别统计
    class_count = y[y.columns[0]].value_counts()
    #极大似然估计：类先验概率
    class_prior = class_count/len(y)
    #类条件概率：字典初始化
    prior = dict()
    #遍历计算类条件概率
    #遍历计算特征
    for col in X.columns:
        #遍历类别
        for j in classes:
            p_x_y = X[(y==j).values][col].value_counts()
            for i in p_x_y.index:
                prior[(col, i, j)] = p_x_y[i]/class_count[j]
    return classes, class_prior, prior

In [16]:
X[(y==1).values]

,x1,x2
2,1,M
3,1,S
7,2,M
8,2,L
9,2,L
10,3,L
11,3,M
12,3,M
13,3,L


In [13]:
X.columns

Index(['x1', 'x2'], dtype='object')

In [17]:
X[(y==1).values]['x1']

2     1
3     1
7     2
8     2
9     2
10    3
11    3
12    3
13    3
Name: x1, dtype: int64

In [22]:
t = X[(y==1).values]['x1'].value_counts()
t.index

Int64Index([3, 2, 1], dtype='int64')

In [25]:
nb_fit(X,y)

(array([-1,  1]),
  1    0.6
 -1    0.4
 Name: y, dtype: float64,
 {('x1', 1, -1): 0.5,
  ('x1', 2, -1): 0.3333333333333333,
  ('x1', 3, -1): 0.16666666666666666,
  ('x1', 3, 1): 0.4444444444444444,
  ('x1', 2, 1): 0.3333333333333333,
  ('x1', 1, 1): 0.2222222222222222,
  ('x2', 'S', -1): 0.5,
  ('x2', 'M', -1): 0.3333333333333333,
  ('x2', 'L', -1): 0.16666666666666666,
  ('x2', 'M', 1): 0.4444444444444444,
  ('x2', 'L', 1): 0.4444444444444444,
  ('x2', 'S', 1): 0.1111111111111111})

In [26]:
classes, class_prior, prior = nb_fit(X, y)
print(classes, class_prior, prior)

[-1  1]  1    0.6
-1    0.4
Name: y, dtype: float64 {('x1', 1, -1): 0.5, ('x1', 2, -1): 0.3333333333333333, ('x1', 3, -1): 0.16666666666666666, ('x1', 3, 1): 0.4444444444444444, ('x1', 2, 1): 0.3333333333333333, ('x1', 1, 1): 0.2222222222222222, ('x2', 'S', -1): 0.5, ('x2', 'M', -1): 0.3333333333333333, ('x2', 'L', -1): 0.16666666666666666, ('x2', 'M', 1): 0.4444444444444444, ('x2', 'L', 1): 0.4444444444444444, ('x2', 'S', 1): 0.1111111111111111}


In [27]:
X_test = {'x1': 2, 'x2': 'S'}

In [28]:
classes, class_prior, prior = nb_fit(X, y)
#定义朴素贝叶斯预测函数
def predict(X_test):
    '''
    输入：
    X_test:测试输入，字典格式
    输出：
    classes[np.argmax(res)];类别结果1/-1
    '''
    #结果列表初始化
    res = []
    #遍历样本类别
    for c in classes:
        #获取当前类先验概率
        p_y = class_prior[c]
        #初始化类条件概率
        p_x_y = 1
        #历遍字典的每个元素
        for i in X_test.items():
            #似然函数：类条件概率连乘
            p_x_y *= prior[tuple(list(i)+[c])]
        res.append(p_y*p_x_y)
    return classes[np.argmax(res)]

In [37]:
res = []
    #遍历样本类别
for c in classes:
        #获取当前类先验概率
        p_y = class_prior[c]
        #初始化类条件概率
        p_x_y = 1
        #历遍字典的每个元素
        for i in X_test.items():
            #似然函数：类条件概率连乘
            p_x_y *= prior[tuple(list(i)+[c])]
        res.append(p_y*p_x_y)

In [40]:
X_test.items()

dict_items([('x1', 2), ('x2', 'S')])

In [38]:
res

[0.06666666666666667, 0.02222222222222222]

In [39]:
np.argmax(res)

0

In [29]:
print('测试数据预测类别为：', predict(X_test))

测试数据预测类别为： -1


In [15]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Accuracy of GaussianNB in iris data test:", 
      accuracy_score(y_test, y_pred))

Accuracy of GaussianNB in iris data test: 0.9466666666666667
